In [2]:
import os
# os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")
from dotenv import load_dotenv

In [3]:
import google.generativeai as genai
from langchain_google_genai import ChatGoogleGenerativeAI
load_dotenv()
genai.configure(api_key=os.getenv("GOOGLE_API_KEY"))

In [4]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.vectorstores import FAISS
from langchain.text_splitter import CharacterTextSplitter

In [5]:
doc_reader = PyPDFLoader("The_3_Alarms.pdf")
# doc_reader = PyPDFLoader("doc.pdf")
doc_reader

In [6]:
pages = doc_reader.load_and_split()
pages[0]

Document(page_content='THE 3 ALARMSA Simple System to Transform Your Health, Wealth, and Relationships ForeverERIC PARTAKER', metadata={'source': 'The_3_Alarms.pdf', 'page': 1})

In [7]:
text_splitter = CharacterTextSplitter(
    separator="\n",
    chunk_size=1500,
    chunk_overlap=200,
    length_function=len,
    is_separator_regex=False,
)

In [8]:
texts = text_splitter.split_documents(pages)
print(texts[0])

page_content='THE 3 ALARMSA Simple System to Transform Your Health, Wealth, and Relationships ForeverERIC PARTAKER' metadata={'source': 'The_3_Alarms.pdf', 'page': 1}


In [9]:
model = ChatGoogleGenerativeAI(model="gemini-pro",temperature=0.5)

In [11]:
model.invoke("Tell me the overview of the 3 alarms book in brief.")

AIMessage(content='**The 3 Alarms: Managing the Complexity of Modern Cyber Threats**\n\n**Overview:**\n\n"The 3 Alarms" is a comprehensive book that provides a deep dive into the evolving landscape of cyber threats and the challenges faced by organizations in managing them. It presents a framework based on the concept of three "alarms" that signal the need for organizations to take action to protect themselves from cyberattacks.\n\n**The 3 Alarms:**\n\n* **Alarm 1: The Perimeter is Broken**\n   - Discusses the challenges of securing traditional network perimeters and the need for organizations to adopt a "zero trust" approach.\n* **Alarm 2: The Data is Compromised**\n   - Explores the risks associated with data breaches and the importance of data protection strategies.\n* **Alarm 3: The Business is Impacted**\n   - Examines the impact of cyberattacks on business operations and the need for organizations to develop resilience plans.\n\n**Key Concepts:**\n\n* **Complexity and Interconnec

In [ ]:
"""The 3 Alarms: Managing the Complexity of Modern Cyber Threats**\n\n**Overview:**\n\n"
The 3 Alarms" is a comprehensive book that provides a deep dive into the evolving landscape of cyber threats and 
the challenges faced by organizations in managing them. It presents a framework based on the concept of three "al
arms" that signal the need for organizations to take action to protect themselves from cyberattacks.\n\n**The 3 A
larms:**\n\n* **Alarm 1: The Perimeter is Broken**\n   - Discusses the challenges of securing traditional network peri
meters and the need for organizations to adopt a "zero trust" approach.\n* **Alarm 2: The Data is Compromised**\n   - 
Explores the risks associated with data breaches and the importance of data protection strategies.\n* **Alarm 3: The B
usiness is Impacted**\n   - Examines the impact of cyberattacks on business operations and the need for organizations 
to develop resilience plans.\n\n**Key Concepts:**\n\n* **Complexity and Interconnectedness:** The book emphasizes the 
increasing complexity and interconnectedness of modern IT environments, which makes organizations more vulnerable to 
cyber threats.\n* **Cyber Resilience:** It stresses the importance of building cyber resilience by adopting a proactive
and multi-layered approach to cybersecurity.\n* **Human Factor:** The book acknowledges the role of the human factor in
cybersecurity incidents and provides guidance on mitigating these risks.\n* **Integrated Approach:** It advocates for
an integrated approach to cybersecurity that involves collaboration between IT, security, and business stakeholders.\n\n**Target Audience:**\n\n"The 3 Alarms" is intended for cybersecurity professionals, IT managers, business leaders, and anyone interested in understanding the challenges of moder
n cyber threats and developing effective cybersecurity strategies.')"""